In [73]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [74]:
# Load the data
fees_df = pd.read_csv('extract - fees - data analyst - .csv')
cash_requests_df = pd.read_csv('extract - cash request - data analyst.csv')

# Basic data inspection

print(cash_requests_df.head())
print(cash_requests_df.info())
print(cash_requests_df.describe())


print(fees_df.head())
print(fees_df.info())
print(fees_df.describe())

print("Fees data loaded: {} rows, {} columns".format(fees_df.shape[0], fees_df.shape[1]))
print("Cash requests data loaded: {} rows, {} columns".format(cash_requests_df.shape[0], cash_requests_df.shape[1]))

     id  amount    status                     created_at  \
0     5   100.0  rejected  2019-12-10 19:05:21.596873+00   
1    70   100.0  rejected   2019-12-10 19:50:12.34778+00   
2     7   100.0  rejected   2019-12-10 19:13:35.82546+00   
3    10    99.0  rejected  2019-12-10 19:16:10.880172+00   
4  1594   100.0  rejected  2020-05-06 09:59:38.877376+00   

                      updated_at  user_id                   moderated_at  \
0   2019-12-11 16:47:42.40783+00    804.0  2019-12-11 16:47:42.405646+00   
1  2019-12-11 14:24:22.900054+00    231.0  2019-12-11 14:24:22.897988+00   
2  2019-12-11 09:46:59.779773+00    191.0  2019-12-11 09:46:59.777728+00   
3  2019-12-18 14:26:18.136163+00    761.0  2019-12-18 14:26:18.128407+00   
4   2020-05-07 09:21:55.34008+00   7686.0  2020-05-07 09:21:55.320193+00   

   deleted_account_id             reimbursement_date  \
0                 NaN  2020-01-09 19:05:21.596363+00   
1                 NaN   2020-01-09 19:50:12.34778+00   
2             

In [75]:
#  DATA QUALITY ANALYSIS
# Function to analyze data quality
def analyze_data_quality(df, name):

    # Missing values
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)  #new:this is to get 2digit after the . 
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    print("\nMissing Values:")
    print(missing_df[missing_df['Missing Count'] > 0])  #new:only to show useful info about missing value
    
    # Data types
    print("\nData Types:")                            #new: \n
    print(df.dtypes)
    
    # Duplicates
    duplicates = df.duplicated().sum()
    print("\nDuplicate Rows: {}".format(duplicates))
    
    return missing_df

# Analyze both datasets
fees_missing = analyze_data_quality(fees_df, "Fees")
requests_missing = analyze_data_quality(cash_requests_df, "Cash Requests")


Missing Values:
                 Missing Count  Missing %
cash_request_id              4       0.02
category                 18865      89.57
paid_at                   5530      26.26
from_date                13295      63.13
to_date                  13295      63.13

Data Types:
id                   int64
cash_request_id    float64
type                object
status              object
category            object
total_amount       float64
reason              object
created_at          object
updated_at          object
paid_at             object
from_date           object
to_date             object
charge_moment       object
dtype: object

Duplicate Rows: 0

Missing Values:
                            Missing Count  Missing %
user_id                              2103       8.77
moderated_at                         7935      33.10
deleted_account_id                  21866      91.22
cash_request_received_date           7681      32.04
money_back_date                      7427      30.98

In [76]:
# DATA PREPROCESSING
# Convert date columns to datetime
date_columns_fees = ['created_at', 'updated_at', 'paid_at', 'from_date', 'to_date']
date_columns_requests = ['created_at', 'updated_at', 'moderated_at', 'reimbursement_date', 
                        'money_back_date', 'reco_creation', 'reco_last_update']

for col in date_columns_fees:
    if col in fees_df.columns:
        fees_df[col] = pd.to_datetime(fees_df[col], errors='coerce')
        
for col in date_columns_requests:
    if col in cash_requests_df.columns:
        cash_requests_df[col] = pd.to_datetime(cash_requests_df[col], errors='coerce')

# Handle missing user_id by replacing those records with "nan"
original_count = len(cash_requests_df)
missing_count = cash_requests_df['user_id'].isna().sum()
cash_requests_df['user_id'] = cash_requests_df['user_id'].fillna("nan")
print(f"Replaced {missing_count} missing user_id values with 'nan'")            #is this normal 0 number?
print(f"Total records processed: {original_count}") 

Replaced 2103 missing user_id values with 'nan'
Total records processed: 23970


In [77]:
#EXPLORATORY DATA ANALYSIS
# Cash Requests Analysis
fig_eda = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Cash Request Status Distribution', 'Amount Distribution',
                    'Cash Requests Over Time', 'Fee Types Distribution'),
    specs=[[{"type": "bar"}, {"type": "histogram"}],
           [{"type": "scatter"}, {"type": "bar"}]]
)

# Status distribution
status_counts = cash_requests_df['status'].value_counts()
fig_eda.add_trace(
    go.Bar(x=status_counts.index, y=status_counts.values, name='Status'),
    row=1, col=1
)

# Amount distribution
fig_eda.add_trace(
    go.Histogram(x=cash_requests_df['amount'], nbinsx=50, name='Amount'),
    row=1, col=2
)

# Requests over time
monthly_requests = cash_requests_df.groupby(cash_requests_df['created_at'].dt.to_period('M')).size()
fig_eda.add_trace(
    go.Scatter(x=monthly_requests.index.astype(str), y=monthly_requests.values, 
               mode='lines+markers', name='Monthly Requests'),
    row=2, col=1
)

# Fee types distribution
fee_types = fees_df['type'].value_counts()
fig_eda.add_trace(
    go.Bar(x=fee_types.index, y=fee_types.values, name='Fee Types'),
    row=2, col=2
)

fig_eda.update_layout(height=800, title_text="Exploratory Data Analysis Overview", showlegend=False)
fig_eda.show()

# Print summary statistics
print("\nCash Requests Summary Statistics:")
print(cash_requests_df[['amount']].describe().round(2) )

print("\nFees Summary Statistics:")
print(fees_df[['total_amount']].describe().round(2) )


Cash Requests Summary Statistics:
         amount
count  23970.00
mean      82.72
std       26.53
min        1.00
25%       50.00
50%      100.00
75%      100.00
max      200.00

Fees Summary Statistics:
       total_amount
count      21061.00
mean           5.00
std            0.03
min            5.00
25%            5.00
50%            5.00
75%            5.00
max           10.00


In [78]:
# Find first cash request for each user
user_first_request = cash_requests_df.groupby('user_id')['created_at'].min().reset_index()
user_first_request.columns = ['user_id', 'first_request_date']
user_first_request['cohort'] = user_first_request['first_request_date'].dt.to_period('M')

# Merge cohort information back to cash requests
cash_requests_with_cohort = cash_requests_df.merge(user_first_request, on='user_id', how='left')

# Calculate months since first request
cash_requests_with_cohort['months_since_first'] = (
    (cash_requests_with_cohort['created_at'].dt.to_period('M') - 
     cash_requests_with_cohort['cohort']).apply(lambda x: x.n if pd.notna(x) else 0)
)

# Merge fees with cash requests to get cohort information
fees_with_cohort = fees_df.merge(
    cash_requests_with_cohort[['id', 'user_id', 'cohort', 'first_request_date']],
    left_on='cash_request_id',
    right_on='id',
    how='left'
)

print(f"Created cohorts for {user_first_request['user_id'].nunique()} unique users")
print(f"Cohorts range from {user_first_request['cohort'].min()} to {user_first_request['cohort'].max()}")

Created cohorts for 10799 unique users
Cohorts range from 2019-11 to 2020-11


# Q1: Frequency (avg transactions per user)

In [79]:
frequency = cash_requests_with_cohort.groupby(['cohort', 'months_since_first', 'user_id']).size()
print(frequency.groupby(['cohort', 'months_since_first']).mean().unstack(fill_value=0).round(2))


months_since_first    0     1     2     3     4     5     6     7    8   \
cohort                                                                    
2019-11             1.00  1.00  0.00  0.00  0.00  1.00  1.00  0.00  0.0   
2019-12             1.40  1.74  1.62  1.81  2.24  2.75  7.33  9.53  6.3   
2020-01             1.12  1.11  1.07  1.30  1.17  1.04  1.04  1.05  1.1   
2020-02             1.07  1.09  1.17  1.07  1.11  1.03  1.25  1.08  1.2   
2020-03             1.13  1.11  1.28  1.05  1.13  1.07  1.04  1.11  1.0   
2020-04             1.08  1.26  1.23  1.15  1.30  1.18  1.24  0.00  0.0   
2020-05             1.11  1.21  1.20  1.22  1.22  1.11  1.00  0.00  0.0   
2020-06             1.24  1.33  1.25  1.18  1.15  1.00  0.00  0.00  0.0   
2020-07             1.34  1.33  1.17  1.12  1.00  0.00  0.00  0.00  0.0   
2020-08             1.36  1.18  1.14  1.00  0.00  0.00  0.00  0.00  0.0   
2020-09             1.12  1.07  1.00  0.00  0.00  0.00  0.00  0.00  0.0   
2020-10             1.05 

# Q 2: INCIDENT RATE

In [80]:

# identify incident fees
incident_fees = fees_with_cohort[fees_with_cohort['type'] == 'incident']
incident_data = incident_fees.merge(
    cash_requests_with_cohort[['id', 'months_since_first']], 
    left_on='cash_request_id', right_on='id'
)
# Calculate rates
total_requests = cash_requests_with_cohort.groupby(['cohort', 'months_since_first']).size()
incidents = incident_data.groupby(['cohort', 'months_since_first']).size()
print((incidents / total_requests * 100).unstack(fill_value=0).round(2))


months_since_first     0      1      2      3      4      5      6      7   \
cohort                                                                       
2019-11               NaN    NaN   0.00   0.00   0.00    NaN    NaN   0.00   
2019-12               NaN    NaN    NaN    NaN    NaN   1.63  18.18  12.67   
2020-01               NaN    NaN    NaN    NaN   3.70  28.00  35.71   9.09   
2020-02               NaN    NaN    NaN    NaN  14.29  35.14  11.11    NaN   
2020-03               NaN    NaN    NaN  37.21  17.65  16.67   7.41    NaN   
2020-04               NaN   7.63  36.09  28.87  21.21   7.69    NaN   0.00   
2020-05              7.33  24.09  27.31  14.22   8.33   0.61    NaN   0.00   
2020-06             20.33  16.20  15.30  11.13   1.13    NaN   0.00   0.00   
2020-07             18.61  17.76  10.78   1.05    NaN   0.00   0.00   0.00   
2020-08             12.86   8.54   0.72    NaN   0.00   0.00   0.00   0.00   
2020-09              9.59   0.99    NaN   0.00   0.00   0.00   0

#Q3: REVENUE GENERATED BY COHORT


In [82]:
# Filter paid fees and calculate revenue by cohort/month
paid_fees = fees_with_cohort[fees_with_cohort['status'] == 'accepted'].copy()
print(f"Found {len(paid_fees)} accepted/paid fees")

if paid_fees['paid_at'].isna().all():
    print("Using created_at instead of paid_at")
    paid_fees['fee_month'] = paid_fees['created_at'].dt.to_period('M')
else:
    paid_fees['fee_month'] = paid_fees['paid_at'].dt.to_period('M')

# Calculate months since cohort
paid_fees['months_since_cohort'] = (
    (paid_fees['fee_month'] - paid_fees['cohort'])
    .apply(lambda x: x.n if pd.notna(x) else 0)
)

# Sum revenue
revenue = paid_fees.groupby(['cohort', 'months_since_cohort'])['total_amount'].sum()
revenue_table = revenue.unstack(fill_value=0)

print(revenue_table)

Found 14841 accepted/paid fees
months_since_cohort      0        1       2       3       4       5      6   \
cohort                                                                        
2019-11                 0.0      0.0     0.0     0.0     0.0     0.0    0.0   
2019-12               195.0      0.0     0.0     0.0     0.0     0.0   75.0   
2020-01                20.0      0.0     0.0     0.0     0.0    10.0   55.0   
2020-02                40.0      0.0     0.0     0.0    20.0    30.0   45.0   
2020-03                45.0      0.0     0.0    20.0    55.0    40.0  100.0   
2020-04               115.0      0.0    45.0   125.0   115.0   290.0  425.0   
2020-05               280.0    100.0   355.0   330.0   675.0   955.0  600.0   
2020-06              1185.0   1270.0  1080.0  2690.0  3735.0  2425.0  705.0   
2020-07               825.0   1090.0  2390.0  3275.0  2240.0   505.0  175.0   
2020-08               355.0   1395.0  1705.0  1180.0   215.0   115.0   15.0   
2020-09              

# Q4: NEW RELEVANT METRIC -